In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [5]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.55, 1.6, 1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.5, 0.6, 0.7, 0.9)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312      1.0 -7.510005e-06                  NaN           NaN
0.000312      NaN           NaN                  1.0 -1.189558e-05
0.000750      2.0 -6.919919e-06                  2.0 -1.193219e-05
0.001052      3.0 -5.672550e-06                  3.0 -1.117217e-05
0.001476      4.0 -4.345040e-06                  4.0 -1.073208e-05
0.002070      5.0 -2.961614e-06                  5.0 -1.026884e-05
0.002904      6.0 -1.508665e-06                  6.0 -9.779286e-06
0.004074      7.0  1.312035e-08                  7.0 -9.267786e-06
0.005714      8.0  1.605370e-06                  8.0 -8.723377e-06
0.008015      9.0  3.261757e-06                  9.0 -8.156740e-06
0.011243     10.0  4.993077e-06                 10.0 -7.570547e-06
0.015771     11.0  6.800780e-06                 11.0 -6.975937e-06
0.022122     12.0  8.687378e-06                 12.0 -6.344651e-06
0.031031     13.0  1.066664e-05                 13.0 -5.677323e-06
0.043528     14.0  1.330962e-05                 14.0 -4.808824e-06
0.061057     15.0  2.370610e-05                 15.0 -1.184954e-06
0.085645     16.0  5.040278e-05                 16.0  8.399839e-06
0.120136     17.0  9.475622e-05                 17.0  2.460239e-05
0.168516     18.0  1.584078e-04                 18.0  4.805002e-05
0.236378     19.0  2.476754e-04                 19.0  8.103284e-05
0.331549     20.0  3.710050e-04                 20.0  1.265092e-04
0.465100     21.0  5.390742e-04                 21.0  1.899001e-04
0.652400     22.0  7.402300e-04                 22.0  2.705200e-04
0.915100     23.0  8.189034e-04                 23.0  3.038890e-04
1.283650     24.0  7.399414e-04                 24.0  2.750312e-04
1.800600     25.0  6.138147e-04                 25.0  2.275085e-04
2.525700     26.0  4.694976e-04                 26.0  1.728010e-04
3.542800     27.0  3.464878e-04                 27.0  1.259806e-04
4.969550     28.0  2.503133e-04                 28.0  8.939525e-05
6.970850     29.0  1.758310e-04                 29.0  6.082963e-05
9.778100     30.0  1.186502e-04                 30.0  3.867906e-05
13.715850    31.0  7.520215e-05                 31.0  2.154874e-05
19.239350    32.0  4.261063e-05                 32.0  8.359665e-06
26.987250    33.0  1.836887e-05                 33.0 -1.777913e-06
37.855300    34.0  1.454620e-07                 34.0 -9.670021e-06
53.100050    35.0 -1.608919e-05                 35.0 -1.734580e-05
73.887500    36.0 -2.683766e-05                 36.0 -2.338990e-05
97.662500    37.0 -3.027027e-05                 37.0 -2.595311e-05
121.437500   38.0 -3.000015e-05                 38.0 -2.677513e-05
145.212500   39.0 -2.554705e-05                 39.0 -2.543533e-05
168.987500   40.0 -1.755069e-05                 40.0 -2.293660e-05
192.762500   41.0 -8.247865e-07                 41.0 -2.045306e-05
216.537500   42.0  7.633501e-05                 42.0 -7.251064e-07
240.312500   43.0  3.443560e-04                 43.0  1.260379e-04
264.087500   44.0  1.010529e-03                 44.0  5.178652e-04
287.862500   45.0  2.350140e-03                 45.0  1.430127e-03
311.637500   46.0  4.644299e-03                 46.0  3.208741e-03
335.412500   47.0  8.088632e-03                 47.0  6.169034e-03
359.187500   48.0  1.256919e-02                 48.0  1.043432e-02
382.962500   49.0  1.710819e-02                 49.0  1.516968e-02
406.737500   50.0  2.238785e-02                 50.0  2.065570e-02
430.512500   51.0  2.837063e-02                 51.0  2.658956e-02
454.287500   52.0  3.504004e-02                 52.0  3.294789e-02
478.062500   53.0  4.241032e-02                 53.0  3.979118e-02
501.837500   54.0  5.043117e-02                 54.0  4.733163e-02
525.612500   55.0  5.922359e-02                 55.0  5.583461e-02
549.387500   56.0  6.874208e-02                 56.0  6.56969

# Fluxes by Layer

CRD                              CLIRAD             \
                         flug          fldg      fnetg       flug       fldg   
pressure     level                                                             
0.000000e+00 1     -32.497554  0.000000e+00 -32.497554        NaN        NaN   
1.000000e-08 1            NaN           NaN        NaN -32.547198   0.000001   
6.244000e-04 2     -32.497554  3.434516e-09 -32.497554 -32.547198   0.000001   
8.759000e-04 3     -32.497554  4.838863e-09 -32.497554 -32.547198   0.000001   
1.228600e-03 4     -32.497554  6.869621e-09 -32.497554 -32.547198   0.000001   
1.723400e-03 5     -32.497554  9.808422e-09 -32.497554 -32.547198   0.000001   
2.417400e-03 6     -32.497554  1.406033e-08 -32.497554 -32.547198   0.000001   
3.390900e-03 7     -32.497554  2.021224e-08 -32.497554 -32.547198   0.000001   
4.756500e-03 8     -32.497554  2.911249e-08 -32.497554 -32.547198   0.000001   
6.672000e-03 9     -32.497554  4.198726e-08 -32.497554 -32.547198   0.000001   
9.358900e-03 10    -32.497554  6.061046e-08 -32.497554 -32.547198   0.000001   
1.312780e-02 11    -32.497554  8.754644e-08 -32.497554 -32.547198   0.000001   
1.841450e-02 12    -32.497554  1.265036e-07 -32.497554 -32.547198   0.000001   
2.583020e-02 13    -32.497554  1.828429e-07 -32.497554 -32.547198   0.000001   
3.623230e-02 14    -32.497554  2.643238e-07 -32.497554 -32.547198   0.000001   
5.082340e-02 15    -32.497554  3.831480e-07 -32.497554 -32.547198   0.000001   
7.129060e-02 16    -32.497555  5.739773e-07 -32.497554 -32.547198   0.000001   
1.000000e-01 17    -32.497555  9.227093e-07 -32.497554 -32.547198   0.000002   
1.402710e-01 18    -32.497555  1.585707e-06 -32.497553 -32.547198   0.000002   
1.967600e-01 19    -32.497555  2.841233e-06 -32.497552 -32.547198   0.000002   
2.759970e-01 20    -32.497555  5.206994e-06 -32.497550 -32.547199   0.000003   
3.871000e-01 21    -32.497555  9.641159e-06 -32.497545 -32.547199   0.000005   
5.431000e-01 22    -32.497553  1.792166e-05 -32.497535 -32.547199   0.000009   
7.617000e-01 23    -32.497549  3.285419e-05 -32.497516 -32.547198   0.000015   
1.068500e+00 24    -32.497542  5.557531e-05 -32.497486 -32.547196   0.000024   
1.498800e+00 25    -32.497533  8.483607e-05 -32.497449 -32.547194   0.000036   
2.102400e+00 26    -32.497525  1.200332e-04 -32.497405 -32.547193   0.000051   
2.949000e+00 27    -32.497517  1.598983e-04 -32.497358 -32.547193   0.000069   
4.136600e+00 28    -32.497513  2.041996e-04 -32.497309 -32.547195   0.000088   
5.802500e+00 29    -32.497513  2.532688e-04 -32.497259 -32.547199   0.000110   
8.139200e+00 30    -32.497518  3.074786e-04 -32.497211 -32.547208   0.000136   
1.141700e+01 31    -32.497532  3.672632e-04 -32.497165 -32.547222   0.000164   
1.601470e+01 32    -32.497557  4.331255e-04 -32.497124 -32.547243   0.000197   
2.246400e+01 33    -32.497597  5.057072e-04 -32.497091 -32.547274   0.000235   
3.151050e+01 34    -32.497657  5.856619e-04 -32.497072 -32.547318   0.000277   
4.420010e+01 35    -32.497744  6.730428e-04 -32.497071 -32.547381   0.000326   
6.200000e+01 36    -32.497865  7.598873e-04 -32.497105 -32.547469   0.000377   
8.577500e+01 37    -32.498015  8.346116e-04 -32.497181 -32.547584   0.000426   
1.095500e+02 38    -32.498158  8.923445e-04 -32.497266 -32.547697   0.000466   
1.333250e+02 39    -32.498314  9.631050e-04 -32.497351 -32.547822   0.000515   
1.571000e+02 40    -32.498497  1.074809e-03 -32.497422 -32.547970   0.000592   
1.808750e+02 41    -32.498728  1.256438e-03 -32.497472 -32.548158   0.000715   
2.046500e+02 42    -32.499104  1.630157e-03 -32.497474 -32.548469   0.000969   
2.284250e+02 43    -32.500027  2.767650e-03 -32.497259 -32.549258   0.001756   
2.522000e+02 44    -32.501968  5.678193e-03 -32.496289 -32.550968   0.003821   
2.759750e+02 45    -32.505592  1.214864e-02 -32.493443 -32.554233   0.008545   
2.997500e+02 46    -32.511752  2.492692e-02 -32.486825 -32.559887   0.018226   
3.235250e+02 47    -32.521

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')